In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Transbot_Lib import Transbot
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
# 创建Transbot对象 bot
bot = Transbot()


In [ ]:
# 启动接收数据，只能启动一次，所有读取数据的功能都是基于此方法
bot.create_receive_threading()

## 自动上报数据开关

In [ ]:
# 开启自动发送数据
# enable=True，底层扩展板会每隔40毫秒发送一次数据。enable=False，则不发送。
# forever=True永久保存，=False临时作用。
enable = True
bot.set_auto_report_state(enable, forever=False)

In [ ]:
# 关闭自动发送数据
# enable=True，底层扩展板会每隔40毫秒发送一次数据。enable=False，则不发送。
# forever=True永久保存，=False临时作用。
enable = False
bot.set_auto_report_state(enable, forever=False)

In [ ]:
# 清除单片机自动发送过来的缓存数据
bot.clear_auto_report_data()

## 小车控制速度相关参数设置

In [ ]:
# 控制电机运动
def car_motion(line, angular):
    # 为了避免电机死区，这里增加了判断条件。
    if abs(line) >= 5 or abs(angular) >= 10:
        speed_l = line / 100.0
        speed_a = angular / 100.0
        bot.set_car_motion(speed_l, speed_a)
        return speed_l, speed_a
    else:
        bot.set_car_motion(0, 0)
        return 0, 0


# 创建两个滑块来控制小车的线速度和角速度
interact(car_motion, line=widgets.IntSlider(min=-45,max=45,step=1,value=0), \
         angular=widgets.IntSlider(min=-200,max=200,step=1,value=0));


In [ ]:
# 停止运动
bot.set_car_motion(0, 0)

In [ ]:
# 获取小车线速度和角速度数据
try:
    while True:
        v, a = bot.get_motion_data()
        print("speed:", v, a)
        bot.clear_auto_report_data()
        time.sleep(.1)
except KeyboardInterrupt:
    pass

In [ ]:
# 设置小车运动最小速度限制值（最小值）。默认情况下可不设置。
# 举个例子：当设置了线速度限制值为0.1时，下次调用set_car_motion(0.05, 0)函数时，单片机判断0.05小于0.1，自动把输入0.05设置为0.1，如果是set_car_motion(-0.05, 0)，则会把-0.05设置为-0.1。角速度限制值也是同样的功能。
# line_limit=[0, 0.20]，angular_limit=[0, 1.00]
# forever=True永久保存，=False临时作用。
line_limit = 0.1
angular_limit = 0.5
bot.set_speed_limit(line_limit, angular_limit, forever=False)

## PID相关参数设置

In [ ]:
# PID 参数控制，会影响set_car_motion函数控制小车的运动速度变化情况。默认情况下可不调整。
kp = 0.1
ki = 0.0
kd = 4.0
bot.set_pid_param(kp, ki, kd, forever=False)

In [ ]:
kp, ki, kd = bot.get_motion_pid()
print("PID:", kp, ki, kd)

## 陀螺仪相关参数设置

In [ ]:
# 开启陀螺仪调节方向。
# 由于小车振动以及陀螺仪自身漂移的问题，所以开启陀螺仪调节方向后，不能长时间一直运动，否则会陀螺仪会对小车运动造成干扰，导致一个轮子快一个轮子慢。
# 长时间测试时请将此功能关闭。
# enable=True开启，=False关闭。
# forever=True永久保存，=False临时作用。此参数最好不要调节，保持默认即可。
enable = True
bot.set_imu_adjust(enable, forever=False)

In [ ]:
# 关闭陀螺仪调节方向。
# enable=True开启，=False关闭。
# forever=True永久保存，=False临时作用。此参数最好不要调节，保持默认即可。
enable = False
bot.set_imu_adjust(enable, forever=False)

In [ ]:
state = bot.get_imu_state()
print("imu state:", state)

In [ ]:
# 获取加速度计三轴数据
try:
    while True:
        a_x, a_y, a_z = bot.get_accelerometer_data()
        print("acce:", a_x, a_y, a_z)
        time.sleep(.1)
except KeyboardInterrupt:
    pass

In [ ]:
# 获取陀螺仪三轴数据
try:
    while True:
        g_x, g_y, g_z = bot.get_gyroscope_data()
        print("gyro:", g_x, g_y, g_z)
        time.sleep(.1)
except KeyboardInterrupt:
    pass

## 恢复出厂配置，重置Flash数据。

In [ ]:
# 恢复出厂配置
bot.reset_flash_value()

In [ ]:
# 程序结束后请删除transbot对象，避免在其他程序中使用transbot库造成冲突
del bot